### 본 예제는 optimizer 사용에 있어 사용자가 정의한 gradient를 직접 학습에 적용하는 방법을 익히는 데 초점을 둠.
전반적인 내용은 모두 일치하며 아래 함수가 주된 내용임

* optimizer.compute_gradients(loss, vars) : vars에 포함된 각 variable에 대하여 loss의 gradient 값을 리스트로 리턴함
* optimizer.apply_gradient(grad) : 사용자가 직접 정의한 gradient 값인 grad를 적용하여 파라미터 업데이트를 진행. minimize()의 second part.
* tf.clip_by_value(grad, min, max) : 입력된 grad값과 min, max 범위를 고려하여 grad 값이 범위 밖에 있을 경우, min 또는 max 값으로 대체


### 주의할 점
compute_gradients 함수는 (gradient 값, 대상 variable) pair로 이루어진 리스트를 반환한다.

따라서 사용자가 직접 gradient를 tf.apply_gradients를 통해 지정해 줄 때도, 위의 형식을 맞춰주어야한다 !

In [1]:
# Lab 3 Minimizing Cost
# This is optional
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

# tf Graph Input
X = [1, 2, 3]
Y = [1, 2, 3]

# Set wrong model weights
W = tf.Variable(5.)

# Linear model
hypothesis = X * W

In [2]:
# Manual gradient
gradient = tf.reduce_mean((W * X - Y) * X) * 2

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize: Gradient Descent Magic
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

### 다양한 gradient 직접 적용
아래 부분이 optimizer의 내부 함수를 통해 각 변수별 gradient 값을 계산하고,

계산한 결과를 직접 apply_gradient 를 통해 반영하여 학습하는 부분이다.

apply_gradient를 실행할 경우, 적용한 gradient에 따라 파라미터 업데이트 (학습)이 진행되므로 별도로 optimizer.minimize(cost)를 실행해줄 필요는 없다.

In [5]:
# Get gradients
# gvs = optimizer.compute_gradients(cost, [W]) #원래 gradient 값을 계산하여 반환받는 부분
print gvs
# Optional: modify gradient if necessary
gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs] # gradient의 범위를 지정했을 경우
# Apply gradients
apply_gradients = optimizer.apply_gradients(gvs) #이를 실행하면 파라미터 업데이트 까지 진행됨

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(100):
    print(step, sess.run([gradient, W, gvs]))
    sess.run(apply_gradients)
    # Same as sess.run(train)

[(<tf.Tensor 'gradients_2/mul_grad/tuple/control_dependency_1:0' shape=() dtype=float32>, <tensorflow.python.ops.variables.Variable object at 0x10b373a50>)]
(0, [37.333332, 5.0, [(1.0, 5.0)]])
(1, [37.239994, 4.9899998, [(1.0, 4.9899998)]])
(2, [37.146664, 4.9799995, [(1.0, 4.9799995)]])
(3, [37.053329, 4.9699993, [(1.0, 4.9699993)]])
(4, [36.959991, 4.9599991, [(1.0, 4.9599991)]])
(5, [36.866653, 4.9499989, [(1.0, 4.9499989)]])
(6, [36.773319, 4.9399986, [(1.0, 4.9399986)]])
(7, [36.679989, 4.9299984, [(1.0, 4.9299984)]])
(8, [36.586651, 4.9199982, [(1.0, 4.9199982)]])
(9, [36.493313, 4.9099979, [(1.0, 4.9099979)]])
(10, [36.399979, 4.8999977, [(1.0, 4.8999977)]])
(11, [36.306644, 4.8899975, [(1.0, 4.8899975)]])
(12, [36.213306, 4.8799973, [(1.0, 4.8799973)]])
(13, [36.119968, 4.869997, [(1.0, 4.869997)]])
(14, [36.026638, 4.8599968, [(1.0, 4.8599968)]])
(15, [35.933304, 4.8499966, [(1.0, 4.8499966)]])
(16, [35.839966, 4.8399963, [(1.0, 4.8399963)]])
(17, [35.746628, 4.8299961, [(1.0,

위의 결과를 보면, 우리가 직접 계산한 (manual gradient)의 값이 -1, +1 사이의 범위를 벗어나 실제 적용된 gradient 값은 모두 1임을 알 수 있다.

실제 gradient 크기보다 훨씬 작은 값이 학습에 적용되었기 때문에 학습이 더딘 것을 볼 수 있다.

하지만, 이러한 clipping 방법은 학습에 있어 gradient exploding 또는 vanishing 현상을 1차적으로 방지할 수 있는 가장 기본적인 방법이다.